This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'YXJ9BzqyUGpYZtyvSTyX'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [2]:
# First, import the relevant modules
import requests
import collections

# Answer to Q.1

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X-Carl-Zeiss-Meditec-AFX_X/data.json'
params = dict(api_key=API_KEY, start_date='2017-01-01', end_date='2017-12-31')
r = requests.get(url, params=params)
json_data = r.json()

# Answer to Q.2

In [4]:
print(type(json_data))

<class 'dict'>


In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json_data.keys())
print(json_data['dataset_data'].keys())

dict_keys(['dataset_data'])
dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [6]:
#'data' holds list of lists containing info on dates
prices = json_data['dataset_data']['data']
print(type(prices))
print(prices[0])

<class 'list'>
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


# Answer to Q.3

In [7]:
#Check index of low and high values corresponding to the stock values
data_header = ['Date','Open','High','Low','Close','Change','Traded_Volume','Turnover','Last_Price','Daily_TU','Daily_Turnover']
low_idx, high_idx = data_header.index('Low'), data_header.index('High')


#Create a list of low and high values from nested list 'prices'
low = [i[low_idx] for i in prices]
high = [j[high_idx] for j in prices]

#Check if all values within the list are float values
print(all(isinstance(n, float) for n in low))
print(all(isinstance(n, float) for n in high))

#Print lowest and highest values for the year 2017
print(min(low))
print(max(high))

True
True
33.62
53.54


# Answer to Q.4

In [8]:
#Check if length of low and high values are the same
print(len(low) == len(high))

#Calculate difference
print(max([high[i] - low[i] for i in range(len(low))]))

True
2.8100000000000023


# Answer to Q.5

In [9]:
#Extract list of values for closing price and check if list contains only float values
close_idx = data_header.index('Close')
closing = [i[close_idx] for i in prices]
print(all(isinstance(n, float) for n in closing))

True


In [10]:
#Calculate largest difference between any two sequential days
largest_diff_clos = 0

for i in range(len(closing)):
    if i == 0:
        previous_clos = closing[i]
        continue
    else:
        diff = abs(closing[i] - previous_clos)
        if diff > largest_diff_clos:
            largest_diff_clos = diff
            previous_clos = closing[i]
        else:
            previous_clos = closing[i]

print(largest_diff_clos)

2.559999999999995


In [11]:
#Calculate largest difference between any two days
print(max(closing) - min(closing))

19.03


# Answer to Q.6

In [12]:
#Extract list of values for trading volume and check if list contains only float values
tv_idx = data_header.index('Traded_Volume')
trading_vol = [i[tv_idx] for i in prices]
print(all(isinstance(n, float) for n in trading_vol))

True


In [13]:
#Print average daily trading volume for 2017
print(sum(trading_vol)/len(trading_vol))

89124.33725490196


# Answer to Q.7

In [14]:
#Using test list to test median function
#test = [1,3,4,5,6,7,9]
#len(test)

In [15]:
#Create function that returns median for a list
def median(lst):
    sort = sorted(lst)
    length = len(lst)
    h_m = length//2
    l_m = length//2 - 1
    
    if length % 2 == 0: #List has an even length, median must be calculated for
        median = (sort[h_m] + sort[l_m])/2
        return median
    else:
        median = sort[h_m]
        return median


In [16]:
#Print median of trading volume using defined function 'median'
print(median(trading_vol))

76286.0
